# Machine Learning no Agronegócio: Recomendação de Culturas\n
\n
## Objetivo\n
\n
Este projeto visa aplicar técnicas de Machine Learning para recomendar a cultura mais adequada para plantio, com base em variáveis de solo e clima (N, P, K, temperatura, umidade, pH e chuva). O trabalho segue as diretrizes da Opção 2 do "Programa Ir Além".\n
\n
## 1. Configuração e Carregamento de Dados

In [ ]:
import pandas as pd\n
import numpy as np\n
import matplotlib.pyplot as plt\n
import seaborn as sns\n
from sklearn.model_selection import train_test_split\n
from sklearn.preprocessing import StandardScaler\n
from sklearn.linear_model import LogisticRegression\n
from sklearn.tree import DecisionTreeClassifier\n
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier\n
from sklearn.svm import SVC\n
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix\n
\n
# Configuração de estilo para visualização\n
sns.set_style('whitegrid')\n
plt.rcParams['figure.figsize'] = (12, 8)\n
\n
# Carregamento do dataset\n
try:\n
    # O arquivo foi renomeado para produtos_agricolas.csv conforme o requisito\n
    data = pd.read_csv('../produtos_agricolas.csv')\n
    # O dataset do Kaggle tem as colunas na ordem correta, mas vamos garantir o nome\n
    data.columns = ['N', 'P', 'K', 'temperatura', 'umidade', 'pH', 'chuva', 'label']\n
    print("Dados carregados com sucesso. Primeiras 5 linhas:")\n
    print(data.head())\n
    print("\nInformações do Dataset:")\n
    data.info()\n
except FileNotFoundError:\n
    print("Erro: O arquivo 'produtos_agricolas.csv' não foi encontrado. Certifique-se de que ele está no diretório correto.")\n
    data = None

## 2. Análise Exploratória de Dados (EDA)\n


In [ ]:
# 1. Distribuição das Culturas (Variável Alvo)\n
plt.figure(figsize=(15, 6))\n
sns.countplot(y='label', data=data, palette='viridis')\n
plt.title('Distribuição das Culturas (Label)')\n
plt.xlabel('Contagem')\n
plt.ylabel('Cultura')\n
plt.show()\n
\n
# 2. Distribuição das Variáveis de Nutrientes (N, P, K)\n
fig, axes = plt.subplots(1, 3, figsize=(18, 6))\n
sns.histplot(data['N'], bins=20, kde=True, ax=axes[0], color='skyblue')\n
axes[0].set_title('Distribuição de Nitrogênio (N)')\n
sns.histplot(data['P'], bins=20, kde=True, ax=axes[1], color='salmon')\n
axes[1].set_title('Distribuição de Fósforo (P)')\n
sns.histplot(data['K'], bins=20, kde=True, ax=axes[2], color='gold')\n
axes[2].set_title('Distribuição de Potássio (K)')\n
plt.tight_layout()\n
plt.show()\n
\n
# 3. Distribuição das Variáveis Climáticas (Temperatura, Umidade, Chuva)\n
fig, axes = plt.subplots(1, 3, figsize=(18, 6))\n
sns.histplot(data['temperatura'], bins=20, kde=True, ax=axes[0], color='red')\n
axes[0].set_title('Distribuição de Temperatura')\n
sns.histplot(data['umidade'], bins=20, kde=True, ax=axes[1], color='blue')\n
axes[1].set_title('Distribuição de Umidade')\n
sns.histplot(data['chuva'], bins=20, kde=True, ax=axes[2], color='green')\n
axes[2].set_title('Distribuição de Chuva')\n
plt.tight_layout()\n
plt.show()\n
\n
# 4. Relação entre pH e Cultura (Boxplot)\n
plt.figure(figsize=(15, 6))\n
sns.boxplot(x='label', y='pH', data=data, palette='coolwarm')\n
plt.title('pH por Tipo de Cultura')\n
plt.xticks(rotation=45, ha='right')\n
plt.tight_layout()\n
plt.show()\n
\n
# 5. Mapa de Calor da Correlação\n
plt.figure(figsize=(10, 8))\n
sns.heatmap(data[['N', 'P', 'K', 'temperatura', 'umidade', 'pH', 'chuva']].corr(), \n
            annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)\n
plt.title('Mapa de Calor da Correlação entre Variáveis')\n
plt.show()

## 3. Identificação do Perfil Ideal de Solo/Clima\n


In [ ]:
# Escolha de 3 culturas: Arroz (rice), Milho (maize) e Algodão (cotton)\n
culturas_escolhidas = ['rice', 'maize', 'cotton']\n
\n
perfil_ideal = data[data['label'].isin(culturas_escolhidas)].groupby('label').agg({\n
    'N': 'mean',\n
    'P': 'mean',\n
    'K': 'mean',\n
    'temperatura': 'mean',\n
    'umidade': 'mean',\n
    'pH': 'mean',\n
    'chuva': 'mean'\n
}).reset_index()\n
\n
print("Perfil Ideal de Solo/Clima para as 3 Culturas Escolhidas:")\n
print(perfil_ideal.round(2).to_markdown(index=False))

## 4. Desenvolvimento e Avaliação de Modelos Preditivos\n


In [ ]:
# Preparação dos Dados\n
X = data[['N', 'P', 'K', 'temperatura', 'umidade', 'pH', 'chuva']]\n
y = data['label']\n
\n
# Divisão em treino e teste\n
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)\n
\n
# Escalonamento (necessário para alguns modelos como SVM e Regressão Logística)\n
scaler = StandardScaler()\n
X_train_scaled = scaler.fit_transform(X_train)\n
X_test_scaled = scaler.transform(X_test)\n
\n
# Definição dos 5 Modelos\n
models = {\n
    "Regressão Logística": LogisticRegression(max_iter=1000, random_state=42),\n
    "Árvore de Decisão": DecisionTreeClassifier(random_state=42),\n
    "Random Forest": RandomForestClassifier(random_state=42),\n
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),\n
    "Support Vector Machine (SVM)": SVC(random_state=42)\n
}\n
\n
results = []\n
\n
# Treinamento e Avaliação\n
for name, model in models.items():\n
    # Modelos baseados em árvore não precisam de dados escalonados\n
    if name in ["Árvore de Decisão", "Random Forest", "Gradient Boosting"]:\n
        model.fit(X_train, y_train)\n
        y_pred = model.predict(X_test)\n
    else:\n
        model.fit(X_train_scaled, y_train)\n
        y_pred = model.predict(X_test_scaled)\n
    \n
    accuracy = accuracy_score(y_test, y_pred)\n
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)\n
    \n
    results.append({\n
        'Modelo': name,\n
        'Acurácia': accuracy,\n
        'Precisão Média': report['macro avg']['precision'],\n
        'Recall Médio': report['macro avg']['recall'],\n
        'F1-Score Médio': report['macro avg']['f1-score']\n
    })\n
    \n
    print(f"\n--- Relatório de Classificação para {name} ---")\n
    print(classification_report(y_test, y_pred, zero_division=0))\n
    \n
    # Exemplo de Matriz de Confusão para o melhor modelo (Random Forest)\n
    if name == "Random Forest":\n
        cm = confusion_matrix(y_test, y_pred)\n
        plt.figure(figsize=(15, 10))\n
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',\n
                    xticklabels=model.classes_, yticklabels=model.classes_)\n
        plt.title('Matriz de Confusão - Random Forest')\n
        plt.xlabel('Previsto')\n
        plt.ylabel('Real')\n
        plt.show()\n
\n
# Avaliação Comparativa dos Modelos\n
results_df = pd.DataFrame(results).sort_values(by='Acurácia', ascending=False)\n
print("\n## 5. Avaliação Comparativa dos Modelos")\n
print(results_df.round(4).to_markdown(index=False))